In [1]:
!pip install datasets transformers evaluate wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict
from datasets import Dataset
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from transformers import DataCollatorWithPadding
import wandb
from datetime import datetime
import os

In [3]:
def load_dataset(fields: List[str], files: Dict[str, str], mapping: Dict[str, int]) -> pd.DataFrame:
    dataset = {}
    
    def encode_label(label: str):
        return mapping[label]

    def load_data(file: str):
        df = pd.read_json(file, lines=True)

        data = []
        for _, i in df.iterrows():
            text = ""
            for field in fields:
                if isinstance(i[field], list):
                    text += ' '.join(i[field])
                elif isinstance(field, str):
                    text += i[field]
                else:
                    raise NotImplemented

            data.append({
                "text": text,
                "label": encode_label(i["tags"][0])})
        return data


    for split in list(files.keys()):
        dataset[split] = load_data(files[split])

    return dataset

In [4]:
def preprocess_dataset(dataset: List[Dict[str, List[str]]], model_base: str, model_name: str):
    if model_base.startswith("roberta"):
        from transformers import RobertaTokenizer
        tokenizer = RobertaTokenizer.from_pretrained(model_name)

    elif model_base.startswith("bert"):
        from transformers import AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)     

    else:
        raise NotImplementedError    

    # create tokanizer function with correct tokenizer
    def preprocess_function(sample):
        return tokenizer(sample["text"], truncation=True) 

    # process slices
    dataset_tokenized = {}
    for slice in list(dataset.keys()):
        slice_dataset = Dataset.from_list(dataset[slice])
        slice_tokenized = slice_dataset.map(preprocess_function, batched=True)
        dataset_tokenized[slice] = slice_tokenized

    return dataset_tokenized

In [5]:
def create_model(model_base: str, model_name: str, file_name: str, dataset):
    # Get num labels
    num_label = len(dataset[list(dataset.keys())[0]].unique("label"))

    if model_base.startswith("roberta"):
        from transformers import RobertaTokenizer
        tokenizer = RobertaTokenizer.from_pretrained(model_name)

    elif model_base.startswith("bert"):
        from transformers import AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)     

    else:
        raise NotImplementedError   

    # Metrics
    metric = evaluate.load("f1")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels, average="weighted")
    
    # Model
    training_args = TrainingArguments(
        output_dir="drive/MyDrive/ANLP/checkpoints/"+file_name,
        logging_dir='./LMlogs', 
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,
        weight_decay=0.01,
        report_to = "wandb",
        logging_steps = 10,
        load_best_model_at_end=True,
        metric_for_best_model='loss',
        greater_is_better=False,
        evaluation_strategy='epoch',
        save_strategy='epoch',)


    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_label)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()

    return trainer

In [6]:
def config_name(model, fields, mapping):
    now = datetime.now().strftime("%Y-%m-%d-T%H-%M-%S")
    if sum(mapping.values())> 1:
        type_ = "multiclass"
    else:
        type_ =  [k for k, v in mapping.items() if v == 1][0]
    return model+"-"+str(len(fields))+"_"+type_+"-"+now


In [7]:
# Grid search parameter
models = ["roberta-base", "roberta-news-full"]
field_config = [["postText"], ["postText", "targetTitle"], ["postText", "targetTitle", "targetParagraphs"]]
types = ["multiclass", "one_against_the_others"]


# get confogurations
configs = []
for model in models:
    for fields in field_config:
        for type in types:
            if type == "multiclass":
                mapping = {'passage': 0, 'phrase':1, 'multi':2}
                configs.append((model, fields, mapping))


            elif type == "one_against_the_others":
                classes = ["passage", "phrase", "multi"]
                for class_ in classes:
                    mapping = {}
                    for c in classes:
                        if c == class_:
                            mapping[c] = 1
                        else:
                            mapping[c] = 0
                    configs.append((model, fields, mapping))

In [ ]:
LOGGING = True
MODEL_BASE = "roberta"
PROJECT_NAME = "SemEval23-classification"


data_paths = {
    "train": "drive/MyDrive/ANLP/data/webis-clickbait-22/train.jsonl", 
    "validation":"drive/MyDrive/ANLP/data/webis-clickbait-22/validation.jsonl"}


for model, fields, mapping in configs:
  file_name = config_name(model, fields, mapping)

  # ignore trained configs
  model_file_name = "-".join(file_name.split("-")[:-6])
  models_trained = os.listdir("drive/MyDrive/ANLP/models")
  models_trained = [ "-".join(name.split("-")[:-6]) for name in models_trained]
  if model_file_name not in models_trained:

    print("Start training")
    print("Configs:", model, fields, mapping)
    print("Name:", file_name)

    # Load dataset
    dataset = load_dataset(fields=fields, files=data_paths, mapping=mapping)
    dataset = preprocess_dataset(dataset=dataset, model_base=MODEL_BASE, model_name=model)

    if model.endswith("full"):
        model = "drive/MyDrive/ANLP/models/" + model

    # Logging
    if LOGGING:
        wandb.init(project=PROJECT_NAME, entity="jueri")
        #wandb.run.name = file_name
        wandb.log({
            "model": model,
            "name": file_name,
            "fields": str(fields),
            "mapping": str(mapping)
            })


    # Train
    trainer_trained = create_model(MODEL_BASE, model, file_name, dataset)
    print(trainer_trained.evaluate())
    if LOGGING:
        wandb.finish()


    trainer_trained.save_model(os.path.join('drive/MyDrive/ANLP/models', file_name))  # save model

Start training
Configs: roberta-base ['postText', 'targetTitle'] {'passage': 0, 'phrase': 1, 'multi': 0}
Name: roberta-base-2_phrase-2022-12-21-T17-55-49


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jueri. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

...

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_

Epoch,Training Loss,Validation Loss,F1
1,0.534100,0.566136,0.720527


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to drive/MyDrive/ANLP/checkpoints/roberta-base-3_passage-2022-12-21-T18-36-35/checkpoint-400
Configuration saved in drive/MyDrive/ANLP/checkpoints/roberta-base-3_passage-2022-12-21-T18-36-35/checkpoint-400/config.json
Model weights saved in drive/MyDrive/ANLP/checkpoints/roberta-base-3_passage-2022-12-21-T18-36-35/checkpoint-400/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/ANLP/checkpoints/roberta-base-3_passage-2022-12-21-T18-36-35/checkpoint-400/tokenizer_config.json
Special tokens file saved in drive/MyDrive/ANLP/checkpoints/roberta-base-3_passage-2022-12-21-T18-36-35/checkpoint-400/special_tokens_map.json
